In [2]:
# Our data scraping journey is not over. Now, we want to scrape our pokedex data.
# pokemondb is a good place to grab this data, as it contains almost all the information we're interested in.

# Specifically, we want to get the pokemon's name, types, abilities, pokedex entries, and stats.
# This code can be easily modified to collect other data that you might consider useful in the future

import time
import pandas as pd
import numpy as np
import random as random
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait 
from selenium.webdriver.chrome.options import Options

In [3]:
# Need to download Chrome webdriver, then install it into PATH.
chrome_options = Options()
chrome_options.add_argument("user-agent=Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/74.0.3729.169 Safari/537.36")

driver = webdriver.Chrome(chrome_options = chrome_options)
# Remember to use the proper URL, aka with the https and stuff
# Navigate to where we want to start
driver.get("https://pokemondb.net/pokedex/bulbasaur")

<ipython-input-3-87d4b09f8ab5>:5: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = chrome_options)


In [8]:
# Set up pokedex lists

poke_name = []

types = []
# type1 = []
# type2 = []

abilities = []
# ability1 = []
# ability2 = []
# ability3 = []
pokedex = []

# This will be a list of lists, which we can easily convert into a dataframe
stats = []
# hp = []
# attack = []
# defence = []
# special_attack = []
# special_defence = []
# speed = []

for i in range(5):
    time.sleep(random.random())
    
    name = driver.find_element_by_xpath('//h1').text
    
    # The xpath here searches for a "tr" element that contains a "th" element
    # with the text "Types".
    type_links = driver.find_elements_by_xpath('//*[@class="vitals-table"]/tbody[1]/tr[th/text() ="Type"]//a')
    
    # The xpath here searches for a "tr" element that contains a "th" element
    # with the text "Abilities".
    ability_links = driver.find_elements_by_xpath('//tr[th/text() = "Abilities"]//*[@class="text-muted"]/a')
    pokedex_links = driver.find_elements_by_xpath('//*[@class="cell-med-text"]')

    stat_links = driver.find_elements_by_xpath('//tr/td[@class="cell-num"][1]') # first six elements are the HP-->Speed stats
    

    poke_name.append(name)
    
    if len(type_links) > 2:
        type_links = type_links[:2]
    
    type_list = [i.text.lower() for i in type_links]
    type_list.extend(["none"] * (2 - len(type_links)))
    types.append(type_list)
    
    # Some pokemon have alternate forms - we want to ignore those for now.
    if len(ability_links) > 3:
        ability_links = ability_links[:3]
    
    ability_list = [i.text.lower() for i in ability_links]
    ability_list.extend(["none"] * (3 - len(ability_links)))
    abilities.append(ability_list)
    
    pokedex.append(" ".join([i.text for i in pokedex_links]))
    
    stats.append([i.text for i in stat_links[:6]])
    
#     hp.append(int(stat_links[0].text))
#     attack.append(int(stat_links[1].text))
#     defence.append(int(stat_links[2].text))
#     special_attack.append(int(stat_links[3].text))
#     special_defence.append(int(stat_links[4].text))
#     speed.append(int(stat_links[5].text))
    
    # Click to the next page
    driver.find_element_by_xpath('//*[@class="entity-nav-next"]').click()

In [9]:
# All the data we collected.
lists = [poke_name, types, abilities, pokedex, stats]

# Turn all the lists into dataframes, then concatenate the dataframes into a final, large dataframe.
pokedex_matrix = pd.concat([pd.DataFrame(i) for i in lists], axis = 1)

# Create unique column names
pokedex_matrix.columns = [i for i in range(len(pokedex_matrix.columns))]

# # Drop null columns
# pokedex_matrix_dropped = pokedex_matrix.drop(pokedex_matrix.iloc[:,3:13].columns, axis = 1)

# Final matrix
pokedex_matrix.columns = ["name", "type1", "type2", "ability1", "ability2", "ability3",
                   "pokedex_entry", "hp","attack","defence","sp_attack","sp_defence", "speed"]

display(pokedex_matrix)

,name,type1,type2,ability1,ability2,ability3,pokedex_entry,hp,attack,defence,sp_attack,sp_defence,speed
0,Charizard,fire,flying,blaze,solar power,,Spits fire that is hot enough to melt boulders...,78,84,78,109,85,100
1,Squirtle,water,none,torrent,rain dish,none,"After birth, its back swells and hardens into ...",44,48,65,50,64,43
2,Wartortle,water,none,torrent,rain dish,none,Often hides in water to stalk unwary prey. For...,59,63,80,65,80,58
3,Blastoise,water,,torrent,rain dish,,A brutal POKéMON with pressurized water jets o...,79,83,100,85,105,78
4,Caterpie,bug,none,shield dust,run away,none,Its short feet are tipped with suction pads th...,45,30,35,20,20,45


In [106]:
pokedex_matrix_dropped.to_csv("Full Pokedex.csv")

,0,1,2
0,poison,,
1,poison,,
2,water,none,None
3,water,ice,None
4,ghost,poison,None
